# Download Classification and Manually Edit

In [1]:
import geopandas as gpd
import boto3
import os
import sys
sys.path.insert(0, './tools/')
from addImageCalc import addImageCalc

### User defines the validation site and the directory where files will be downloaded

In [8]:
#Local directory where you want the classified image downloaded
downloadDir = '/Users/mbonnema/Documents/OPERA_calval/DSWx/FakeChip/review/'

In [9]:
#Only specify one. Leave the other as ''. If more than one planet image for given chip, PLANET_ID must be specified
PLANET_ID = ''
SITE_NAME = '2_1'
assert((len(PLANET_ID) == 0) ^ (len(SITE_NAME) == 0))

### Open AWS session and read image and image_calc tables

In [10]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [11]:
bucket_name = 'opera-calval-database-dswx'

In [15]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
imageTable = gpd.read_file(image_table_data)

In [16]:
obj = s3.Object(bucket_name,'image_calc.geojson')
image_table_data = obj.get()['Body']
image_calcs = gpd.read_file(image_table_data)

In [17]:
# This cell will show the number of planet images found for a given chip. If more than one, ensure the printed Planet
# ID matches the planet image used to generate the classification
temp = imageTable[['image_name', 'site_name']]
df_site2image = temp.set_index('site_name')
df_image2site = temp.set_index('image_name')
if not PLANET_ID:
    values = PLANET_ID = df_site2image.loc[SITE_NAME].tolist()
    PLANET_ID = values[0]
    print(f'There was {len(values)} planet images for this chip')
else:
    values = df_image2site.loc[PLANET_ID].tolist()
    SITE_NAME = values[0]
    print(f'There were {len(values)} chips for this planet_image')

(SITE_NAME, PLANET_ID)

There was 1 planet images for this chip


('2_1', '20210928_141837_16_2407')

In [19]:
def downloadImage_calc(row,downloadDir):
    bucket = row.bucket.iloc[0]
    keys = row.s3_keys.iloc[0]
    keys = keys.split(',')
    for key in keys:
        filename = key.split('/')[-1]
        response = s3_client.download_file(bucket,key,downloadDir+filename)

### Search for specific image and classified image for the defined validation site

In [20]:
#This cell shows all classifications of specified planet image
search = image_calcs[image_calcs.image_name == PLANET_ID]
search.head(20)

,bucket,calc_type,calculated_by,image_calc_name,image_name,notes,oversight_level,previous_name,processing_level,public,reviewed_by,s3_keys,upload_date,version,geometry
8,opera-calval-database-dswx,TEST,Matthew Bonnema,20210928_141837_16_2407_classification_v0,20210928_141837_16_2407,This is a fake entry for testing. Delete when ...,None,None,Intermediate,True,None,data/site/2_1/image/20210928_141837_16_2407/im...,20220916_115719,0.0,"POLYGON ((-60.62407 -14.76471, -60.66244 -14.9..."


In [23]:
#This cell selects the classificaiton with the highest 'version' value (i.e. the most recent version)
search = search[search.version==search['version'].max()]
imagecalc_row = search.iloc[[0]]
imagecalc_row.head()

,bucket,calc_type,calculated_by,image_calc_name,image_name,notes,oversight_level,previous_name,processing_level,public,reviewed_by,s3_keys,upload_date,version,geometry
8,opera-calval-database-dswx,TEST,Matthew Bonnema,20210928_141837_16_2407_classification_v0,20210928_141837_16_2407,This is a fake entry for testing. Delete when ...,None,None,Intermediate,True,None,data/site/2_1/image/20210928_141837_16_2407/im...,20220916_115719,0.0,"POLYGON ((-60.62407 -14.76471, -60.66244 -14.9..."


### Download classified image to the specified directory

In [24]:
downloadImage_calc(imagecalc_row,downloadDir)